In [1]:
# stop spark
try:
    spark.stop()
except:
    pass

# Using findspark to find automatically the spark folder
import findspark
findspark.init()

#import numpy
import numpy as np

# init variables
minlon,maxlon,minlat,maxlat=-180.0,180.0,-90.0,90.0

# write 0 to file
f = open('tmp','w')
f.write(str(0)+"\n")
f.close()

In [2]:
# import pyspark packages
from pyspark.sql import SparkSession, types
from pyspark.sql.functions import explode
from pyspark.sql.functions import split, col
from pyspark.sql.functions import last,max

In [3]:
# init spark session
spark = SparkSession.builder.appName("engine").getOrCreate()

In [4]:
# read 2 streams - 1) local socket, 2) Android device
line = spark.readStream.format("socket").option("host", "localhost").option("port", 9998).load()
phone = spark.readStream.format("socket").option("host","192.168.1.143").option("port",9000).load()

In [5]:
# split anddroid device stream to id,timestamp,lon,lat
ppoint = phone.select(explode(split(phone.value," ")).alias("id"))
ppoint = ppoint.withColumn('lat', split(ppoint["id"], '\,')[3].cast(types.FloatType()))
ppoint = ppoint.withColumn('lon', split(ppoint["id"], '\,')[2].cast(types.FloatType()))
ppoint = ppoint.withColumn('timestamp', split(ppoint["id"], '\,')[1].cast(types.LongType()))
ppoint = ppoint.withColumn('id', split(ppoint["id"], '\,')[0])
# get latest lon,lat
ppoint = ppoint.groupby('id').agg(max('timestamp'), last('lat'), last('lon')).toDF('id','time','lat','lon').drop('id','time')

def batchProcess2(sdf, epoch_id):
    # convert microbatch to pandas df
    df = sdf.toPandas()
    if df.empty:
        pass
    else:
        global minlon
        global maxlon
        global minlat
        global maxlat
        # extract the variables from pandas df
        minlon = float(df.iloc[0]['lon'] - 0.5)
        maxlon = float(df.iloc[0]['lon'] + 0.5)
        minlat = float(df.iloc[0]['lat'] - 0.5)
        maxlat = float(df.iloc[0]['lat'] + 0.5)

In [6]:
# split socket stream to lat,lon
point = line.select(explode(split(line.value," ")).alias("lon"))
point = point.withColumn('lat', split(point["lon"], '\,')[1].cast(types.FloatType()))
point = point.withColumn('lon', split(point["lon"], '\,')[0].cast(types.FloatType()))
# count grouped by (lat,lon)
point = point.groupby('lat','lon').count()
def batchProcess1(sdf, epoch_id):
    global minlon
    global maxlon
    global minlat
    global maxlat
    # convert microbatch to pandas df
    df = sdf.toPandas()
    # pandas filter to find data withing range
    df1 = df[(df['lat']>=minlat)&(df['lat']<=maxlat)&(df['lon']>=minlon)&(df['lon']<=maxlon)]
    #print(np.sum(df1['count'].to_numpy()))
    # write sum(count) to file
    f = open('tmp','w')
    f.write(str(np.sum(df1['count'].to_numpy()))+"\n")
    f.close()

In [7]:
# foreach process for the 2 streams, run in background
ppoint.writeStream.outputMode("complete").foreachBatch(batchProcess2).start()
point.writeStream.outputMode("complete").foreachBatch(batchProcess1).start()